In [1]:
!nvcc --version

# !pip install git+https://github.com/huggingface/transformers@21fac7abba2a37fae86106f87fcf9974fd1e3830
!pip install git+https://github.com/huggingface/transformers
!pip install accelerate
!pip install qwen-vl-utils
!pip install datasets
!CUDA_VERSION=cu121
!pip install 'vllm==0.6.1' --extra-index-url https://download.pytorch.org/whl/${CUDA_VERSION}

from google.colab import drive
import os
import json
import re
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from datasets import load_dataset

# Mount Google Drive
drive.mount('/content/drive')

# Create necessary directories in Google Drive
output_dir = "/content/drive/My Drive/World Models"
os.makedirs(output_dir, exist_ok=True)

# Use GPU
if torch.cuda.is_available():
    print("GPU is available and ready to use!")
    print(f"Device name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available. Make sure it's enabled in the runtime settings.")

# Response Generation with vLLM
from vllm import LLM, SamplingParams
from transformers import AutoProcessor
from qwen_vl_utils import process_vision_info

model_save_path = os.path.join(output_dir, "qwen_model")
if os.path.exists(model_save_path):
    print("Loading model from local save path.")
    model_path = model_save_path
else:
    print("Downloading model. This may take some time...")
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto", device_map="auto"
    )
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")
    model.save_pretrained(model_save_path)
    processor.save_pretrained(model_save_path)

print(model_save_path)
print(model_path)

from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_path)
if not hasattr(config, "rope_scaling") or "factor" not in config.rope_scaling:
    config.rope_scaling = {"type": "linear", "factor": 1.0}  # Default
if not hasattr(config, "rope_type"):
    config.rope_type = "yarn"  # Default
if not hasattr(config, "max_model_len"):
    config.max_model_len = 2048  # Adjust per the model

config.save_pretrained(model_path)

llm = LLM(
    model=model_path,
    limit_mm_per_prompt={"image": 10, "video": 10},
    dtype=torch.float16,
)

sampling_params = SamplingParams(
    temperature=0.1,
    top_p=0.001,
    repetition_penalty=1.05,
    max_tokens=256,
    stop_token_ids=[],
)

# Load the MathVista dataset
dataset = load_dataset("AI4Math/MathVista")


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-1j22og66
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-1j22og66
  Resolved https://github.com/huggingface/transformers to commit b2f2977533445c4f62bf58e10b1360e6856e78ce
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10291149 sha256=f6a74f14e0894a0132394bfe4565d875ab023d0f815f04a04f6fe470a3c4ff14
  Stored in directory: /tmp/pip-ephem-wheel-cache-6whhok6p/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempt

Mounted at /content/drive
GPU is available and ready to use!
Device name: NVIDIA A100-SXM4-40GB
Loading model from local save path.
/content/drive/My Drive/World Models/qwen_model
/content/drive/My Drive/World Models/qwen_model
WARNING 01-06 01:58:16 config.py:1650] Casting torch.bfloat16 to torch.float16.
INFO 01-06 01:58:16 llm_engine.py:232] Initializing an LLM engine (v0.6.1) with config: model='/content/drive/My Drive/World Models/qwen_model', speculative_config=None, tokenizer='/content/drive/My Drive/World Models/qwen_model', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, d

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 01-06 01:59:48 model_runner.py:1008] Loading model weights took 4.1277 GB


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.10/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


...

(…)-00000-of-00002-6b81bd7f7e2065e6.parquet:   0%|          | 0.00/358M [00:00<?, ?B/s]

(…)-00001-of-00002-6a611c71596db30f.parquet:   0%|          | 0.00/386M [00:00<?, ?B/s]

Generating testmini split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5141 [00:00<?, ? examples/s]

TypeError: unhashable type: 'slice'

In [10]:
subset_dataset = dataset["testmini"].select(range(100))
# Prepare results storage
responses = {}

# Batch processing parameters
batch_size = 8  # Adjust based on GPU memory and model size

# Start timing
start_time = time.time()

# Initialize processor outside the loop for efficiency
processor = AutoProcessor.from_pretrained(model_path)

# Split dataset into batches
for i in range(0, len(subset_dataset), batch_size):
    batch = subset_dataset.select(range(i, min(i + batch_size, len(subset_dataset))))
    prompts = []
    mm_data_list = []

    # Prepare prompts and multimodal data for the batch
    for example in batch:
        try:
            query = example["query"]
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": example["decoded_image"]},
                        {"type": "text", "text": query},
                    ],
                }
            ]

            prompt = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, _ = process_vision_info(messages)

            mm_data = {}
            if image_inputs is not None:
                mm_data["image"] = image_inputs

            prompts.append(prompt)
            mm_data_list.append(mm_data)
        except KeyError as e:
            print(f"Missing key in example: {e}. Skipping this example.")
            continue

    # Skip batch if no valid prompts
    if not prompts:
        print("No valid prompts in this batch. Skipping batch.")
        continue

    # Batch inference
    outputs = llm.generate(
        [
            {"prompt": prompt, "multi_modal_data": mm_data}
            for prompt, mm_data in zip(prompts, mm_data_list)
        ],
        sampling_params=sampling_params
    )

    # Process the results for the batch
    for idx, example in enumerate(batch):
        try:
            response = outputs[idx].outputs[0].text
            responses[example["pid"]] = {
                "query": example["query"],
                "response": response,
                "question_type": example["question_type"],
                "answer_type": example["answer_type"],
                "choices": example.get("choices"),
                "precision": example.get("precision"),
                "answer": example["answer"],
            }
        except (KeyError, IndexError) as e:
            print(f"Error processing example: {e}. Skipping this result.")
            continue

# End timing
time_cost = time.time() - start_time
print(f"Time cost for generating responses: {time_cost:.2f} seconds")

# Save responses to a file
responses_file = os.path.join(output_dir, "responses_batch.json")
with open(responses_file, "w") as f:
    json.dump(responses, f)

print(f"Response generation completed. Saved to {responses_file}.")

Processed prompts: 100%|██████████| 4/4 [00:00<00:00, 13.11it/s, est. speed input: 5638.92 toks/s, output: 98.40 toks/s]

Time cost for generating responses: 34.30 seconds
Response generation completed. Saved to /content/drive/My Drive/World Models/responses_batch.json.
